Aluno: Lincoln Matheus Rodrigues de Almeida
Matrícula: 201910757

Exercício proposto para o dia 18/10.

In [2]:
# Instalação do Pytorch

!pip install torch
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Importando dependências

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [4]:
# Definindo hiperparâmetros

input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 20
batch_size = 100
lr = 1e-3

In [5]:
# Extraindo dados MNIST

train_data = dsets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = dsets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
# Carregando os dados

train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size, 
                                      shuffle = False)

In [7]:
# Definindo modelo de classe

class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)
  
  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [8]:
# Montando o modelo

net = Net(input_size, hidden_size, num_classes)
if torch.cuda.is_available():
  net.cuda()

In [9]:
# Definindo otimizador e função de perda

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam( net.parameters(), lr=lr)

In [17]:
# Treinando o modelo (precisa ativar a aceleração por GPU)

for epoch in range(num_epochs):
  for i ,(images,labels) in enumerate(train_gen):
    images = Variable(images.view(-1,28*28)).cuda()
    labels = Variable(labels).cuda()
    
    optimizer.zero_grad()
    outputs = net(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()
    
    if (i+1) % 100 == 0:
      print('Epoch [%d/%d], Etapa [%d/%d], Perda: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.data))

Epoch [1/20], Etapa [100/600], Perda: 0.0000
Epoch [1/20], Etapa [200/600], Perda: 0.0001
Epoch [1/20], Etapa [300/600], Perda: 0.0000
Epoch [1/20], Etapa [400/600], Perda: 0.0001
Epoch [1/20], Etapa [500/600], Perda: 0.0000
Epoch [1/20], Etapa [600/600], Perda: 0.0000
Epoch [2/20], Etapa [100/600], Perda: 0.0000
Epoch [2/20], Etapa [200/600], Perda: 0.0000
Epoch [2/20], Etapa [300/600], Perda: 0.0000
Epoch [2/20], Etapa [400/600], Perda: 0.0000
Epoch [2/20], Etapa [500/600], Perda: 0.0000
Epoch [2/20], Etapa [600/600], Perda: 0.0000
Epoch [3/20], Etapa [100/600], Perda: 0.0000
Epoch [3/20], Etapa [200/600], Perda: 0.0000
Epoch [3/20], Etapa [300/600], Perda: 0.0000
Epoch [3/20], Etapa [400/600], Perda: 0.0000
Epoch [3/20], Etapa [500/600], Perda: 0.0000
Epoch [3/20], Etapa [600/600], Perda: 0.0000
Epoch [4/20], Etapa [100/600], Perda: 0.0000
Epoch [4/20], Etapa [200/600], Perda: 0.0000
Epoch [4/20], Etapa [300/600], Perda: 0.0000
Epoch [4/20], Etapa [400/600], Perda: 0.0000
Epoch [4/2

In [22]:
# Descrevendo a precisão do modelo

correct = 0
total = 0
for images,labels in test_gen:
  images = Variable(images.view(-1,28*28)).cuda()
  labels = labels.cuda()
  
  output = net(images)
  _, predicted = torch.max(output,1)
  correct += (predicted == labels).sum()
  total += labels.size(0)

print('Precisão geral do modelo: %.3f %%' %((100*correct)/(total+1)))
print (images)
print (labels)

Precisão geral do modelo: 98.420 %
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 7, 8, 9, 7, 8, 6, 4, 1,
        9, 3, 8, 4, 4, 7, 0, 1, 9, 2, 8, 7, 8, 2, 6, 0, 6, 5, 3, 3, 3, 9, 1, 4,
        0, 6, 1, 0, 0, 6, 2, 1, 1, 7, 7, 8, 4, 6, 0, 7, 0, 3, 6, 8, 7, 1, 5, 2,
        4, 9, 4, 3, 6, 4, 1, 7, 2, 6, 5, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2,
        3, 4, 5, 6], device='cuda:0')
